In [19]:
import torch
import torch.nn as nn

In [20]:
import wandb
!wandb login

wandb: Currently logged in as: rajceo2031 (rentio) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [21]:
from dataclasses import dataclass


@dataclass 
class ModelArgs:
    device = 'cuda'
    no_of_neurons = 128
    block_size = 32
    batch_size = 32
    dropout = 0.1
    epoch = 50
    max_lr = 1e-4

In [22]:
torch.set_default_device(ModelArgs.device)

In [23]:
import torch
from torch.utils.data import Dataset, DataLoader


torch.manual_seed(0)


num_samples = 10000 
seq_length = ModelArgs.block_size  
device = ModelArgs.device  


t = torch.linspace(0, 100, num_samples + seq_length, device=device)
# data = torch.sin(t) + 0.1 * torch.randn_like(t)  
data = t

X_tensor = torch.stack([data[i:i+seq_length] for i in range(num_samples)])
y_tensor = data[seq_length:]  # Next value prediction

train_size = int(0.8 * num_samples)

X_train, y_train = X_tensor[:train_size], y_tensor[:train_size]  
X_val, y_val = X_tensor[train_size:], y_tensor[train_size:]  


class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


generator = torch.Generator(device=device)


train_loader = DataLoader(
    train_dataset,
    batch_size=ModelArgs.batch_size,
    shuffle=True,  
    generator=generator,  
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    generator=generator, 
    drop_last=True,
    batch_size=ModelArgs.batch_size,
    shuffle=True, 
)



In [24]:

class InputGate(nn.Module):
    def __init__(self, device, no_of_neurons):
        super().__init__()
        self.it = nn.Linear(in_features= ModelArgs.no_of_neurons + ModelArgs.embedding_dims, out_features=no_of_neurons, device=device, dtype=torch.float32)
        self.ct_bar = nn.Linear(in_features=ModelArgs.no_of_neurons + ModelArgs.embedding_dims, out_features=no_of_neurons, device=device, dtype=torch.float32)

    def forward(self, x, ht_1):
        x = torch.cat([x, ht_1], dim=1)
        _it = torch.nn.functional.sigmoid(self.it(x))
        _ct_bar = torch.nn.functional.tanh(self.ct_bar(x))
        # out = torch.nn.functional.sigmoid(self.linear(x))
        return _it, _ct_bar

In [25]:
class OutputGate(nn.Module):
    def __init__(self, device, no_of_neurons) -> None:
        super().__init__()
        self.linear = nn.Linear(in_features= ModelArgs.no_of_neurons + ModelArgs.embedding_dims, out_features=no_of_neurons, device=device, dtype=torch.float32)
    def forward(self, x, ht_1):
        x = torch.cat([x, ht_1], dim=1)
        out = torch.nn.functional.sigmoid(self.linear(x))
        return out

In [26]:
class ForgetGate(nn.Module):

    def __init__(self, device, no_of_neurons):
        super().__init__()
        self.linear = nn.Linear(in_features=ModelArgs.no_of_neurons + ModelArgs.embedding_dims, out_features=no_of_neurons, device=device, dtype=torch.float32)

    def forward(self, x, ht_1):
        # print("Forgot: ", x.shape)
        # print("Forget: ", ht_1.shape)
        x = torch.cat([x, ht_1], dim=1)
        out = torch.nn.functional.sigmoid(self.linear(x))
        return out

In [27]:
class LSTMBlock(nn.Module):
    def __init__(self, device, no_of_neurons):
        super().__init__()
        self.ip = InputGate(device=device, no_of_neurons=no_of_neurons)
        self.op = OutputGate(device=device, no_of_neurons=no_of_neurons)
        self.forget = ForgetGate(device=device, no_of_neurons=no_of_neurons)
        self.no_of_neurons = no_of_neurons
        self.device=device
        self.linear_layer = nn.Linear(in_features=ModelArgs.no_of_neurons, out_features=ModelArgs.embedding_dims, device=device)
    def forward(self, x, ht_1=None, outputs=None, embeds=None):
        # print("Block: ", x.shape)
        # print("Block: ", ht_1.shape)
        # print("Block: ", ct_1.shape)

        if(ht_1 is None):
          ht_1 = torch.randn( (x.shape[0], self.no_of_neurons), device=self.device, requires_grad=True, dtype=torch.float32)

        ct_1 = torch.randn((x.shape[0], self.no_of_neurons),device=self.device, requires_grad=True, dtype=torch.float32)
        seq_len = x.shape[1]
        # ht_1 = ht_1.unsqueeze(-1)
        if(outputs == None):
            # print("New")
            # print(x)
            outputs = []
            for t in range(seq_len):
                # print("Block: ", x.shape)
                # print("Block: ", ht_1.shape)
                # print("Block: ", ct_1.shape)
                xt = x[:, t, :]
                # print(xt.shape)
                ft = self.forget(xt, ht_1) * ct_1
                it, ct_bar = self.ip(xt , ht_1)
                ct_bar_prime = it * ct_bar
                ct = ft * ct_1 + ct_bar_prime
                ht = self.op(xt, ht_1) * torch.nn.functional.tanh(ct)
                outputs.append(ht)
            return ht, ct, torch.stack(outputs, dim=1)

        elif(outputs is not None):
          # print("Other")
          new_output = []
          for t in range(seq_len):
                # print("Block: ", x.shape)
                # print("Block: ", ht_1.shape)
                # print("Block: ", ct_1.shape)
                xt = outputs[:, t, :]
                # print("Shape: ", xt.shape)
                xt = self.linear_layer(xt)
                # print("After: ", xt.shape)
                ft = self.forget(xt, ht_1) * ct_1
                it, ct_bar = self.ip(xt , ht_1)
                ct_bar_prime = it * ct_bar
                ct = ft * ct_1 + ct_bar_prime
                ht = self.op(xt, ht_1) * torch.nn.functional.tanh(ct)
                new_output.append(ht)
          return ht, ct, torch.stack(new_output, dim=1)

In [38]:
class LSTM(nn.Module):
    def __init__(self, device, no_of_neurons, out_features):
        super().__init__()
        self.block1 = LSTMBlock(device=device, no_of_neurons=no_of_neurons)

        self.embeds_table_de = EmbeddingTable_de(device=device)
    
        self.dropout = nn.Dropout(p=ModelArgs.dropout)
  
        self.output = nn.Linear(in_features=ModelArgs.no_of_neurons, out_features=out_features, device=device, dtype=torch.float32)
    def forward(self, x, outputs=None, initial=None):
      
        if(initial is not None and initial is True):
          x = self.embeds_table_de(x)
       
        if(outputs is not None):
         
          ht, ct, outputs = self.block1(x, outputs=outputs)
        elif(outputs is None):
          ht, ct, outputs = self.block1(x)

        ht = self.dropout(ht)

        out = self.output(ht)
        
        return  out

In [41]:
model = LSTM(device=ModelArgs.device, no_of_neurons=ModelArgs.no_of_neurons, out_features=1)
model = model.to(ModelArgs.device)

In [42]:
!pip install torchinfo

from torchinfo import summary

x = torch.randint(0, 100, (ModelArgs.batch_size,ModelArgs.block_size))  # Random integer between 0 and 100
x = x.to(ModelArgs.device)

summary(model=model,
        input_data=x,
        # input_size=(ModelArgs.batch_size, ModelArgs.block_size, ModelArgs.embeddings_dims),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
LSTM (LSTM)                              [32, 32]             [32, 1]              66,560               True
├─LSTMBlock (block1)                     [32, 32]             [32, 128]            --                   True
│    └─ForgetGate (forget)               [32, 1]              [32, 128]            --                   True
│    │    └─Linear (linear)              [32, 129]            [32, 128]            16,640               True
│    └─InputGate (ip)                    [32, 1]              [32, 128]            --                   True
│    │    └─Linear (it)                  [32, 129]            [32, 128]            16,640               True
│    │    └─Linear (ct_bar)              [32, 129]            [32, 128]            16,640               True
│    └─OutputGate (op)                   [32, 1]              [32, 128]            --                   True
│    │    └─Li

In [17]:
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=ModelArgs.max_lr)

In [ ]:
model.train()
train_losses =  torch.zeros(len(train_loader))
val_losses = torch.zeros(len(val_loader))
wandb.init(
    project='LSTM-From-Scratch'
)
for epoch in range(ModelArgs.epoch):
    
    count = 0
    for X, y in train_loader:
        y_pred = model(X)
        # print(y_pred.shape)
        loss = criterion(y_pred, y)
        train_losses[count] = loss.item()
        # print("Loss: ", loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1
        
    # count = 0
    model.eval()
    count = 0
    for X, y in val_loader:
        y_pred = model(X)
        # print(y_pred.shape)
        loss = criterion(y_pred, y)
        
        # print("Loss: ", loss.item())
        val_losses[count] = loss.item()
        
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        count += 1
    model.train()
    wandb.log({
      "Train Loss": train_losses.mean(),
      "Val Loss": val_losses.mean(),
      "epoch": epoch  
    })
    print("Epoch: ", epoch, "|", "Train Loss: ", train_losses.mean(),  "|", "Val Loss: ", val_losses.mean())
